## Burned area delineation

In [1]:
workflow = dict([('id', 'burned-area-delineation'),
                ('label', 'Burned area delineation'),
                ('doc', 'Burned area delineation using two techniques')])


In [46]:
pre_event = dict([('id', 'pre_event'), 
                  ('label', 'Pre-event product for burned area delineation'),
                  ('doc', 'Pre-event product for burned area delineation'),
                  ('value', '/workspace/ogc-tb16/application-chaining/remote'), 
                  ('type', 'Directory')])

In [47]:
post_event = dict([('id', 'post_event'), 
                  ('label', 'Post-event product for burned area delineation'),
                  ('doc', 'Post-event product for burned area delineation'),
                  ('value', '/workspace/ogc-tb16/application-chaining/remote'), 
                  ('type', 'Directory')])

In [48]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')]) 

In [49]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')]) 

In [50]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from time import sleep
from helpers import *
import shutil

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_burned_area_delineation'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
pre_event_cat = Catalog.from_file(os.path.join(pre_event['value'], 'catalog.json'))
post_event_cat = Catalog.from_file(os.path.join(post_event['value'], 'catalog.json'))

In [53]:
pre_event_cat.describe()

* <Catalog id=catalog-S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808>
  * <Item id=INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808>


In [54]:
post_event_cat.describe()

* <Catalog id=catalog-S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808>
  * <Item id=INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808>


In [55]:
pre_event_item = next(pre_event_cat.get_items())
post_event_item = next(post_event_cat.get_items())

In [56]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr = get_assets(pre_event_item)
asset_post_ndvi, asset_post_ndwi, asset_post_nbr = get_assets(post_event_item)

In [57]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr

('https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NDVI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif',
 'https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NDWI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif',
 'https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NBR_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif')

In [58]:
asset_post_ndvi, asset_post_ndwi, asset_post_nbr

('https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NDVI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif',
 'https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NDWI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif',
 'https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NBR_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif')

In [59]:
scaling_factor = 1/10000

scaling_factor

0.0001

If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [60]:
_mem = '/vsimem/mem.tif'

Process NDVI difference

In [61]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_ndvi),
                         outputType=gdal.GDT_Int16)

width = temp_ds.RasterXSize
height = temp_ds.RasterYSize
geo_transform = temp_ds.GetGeoTransform()
geo_ref = temp_ds.GetProjectionRef()

pre_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [62]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_post_ndvi),
                         outputType=gdal.GDT_Int16)

post_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [63]:
delta_ndvi = ((pre_ndvi - post_ndvi) * scaling_factor).astype(float)

pre_ndvi = post_ndvi = None

Process NDWI difference

In [64]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_ndwi),
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

temp_ds = None

RuntimeError: An error occurred while creating a virtual connection to the DAP server:Error while reading the URL: https://store.terradue.com:443/ogc-tb16/sz1os_67/INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808/NDWI_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808.tif.ver.
The OPeNDAP server returned the following message:
Not Found: The data source or server could not be found.
        Often this means that the OPeNDAP server is missing or needs attention.
        Please contact the server administrator.

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_ndwi),
                         outputType=gdal.GDT_Int16)

post_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_ndwi = ((pre_ndwi - post_ndwi) * scaling_factor).astype(float)

pre_ndwi = post_ndwi = None

Burned area delineation

In [ ]:
burned = np.where(((delta_ndwi  > float(ndwi_threshold['value'])) & (delta_ndvi > float(ndvi_threshold['value']))), 1, 0) 

In [ ]:
output_name = 'DELINEATION'

In [ ]:
default_bands = [{'name': 'dnbr',
                  'common_name': 'dNBR'}, 
                 {'name': 'tvi',
                  'common_name': 'tvi'}]

In [ ]:
temp_name = '_BURNED_NDVI_NDWI_THRESHOLD.tif'
output_name = 'BURNED_NDVI_NDWI_THRESHOLD.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Byte)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(burned),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [ ]:
results = {}

results['B01'] = output_name


relativized burn ratio (RBR) 

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_nbr),
                         outputType=gdal.GDT_Int16)

pre_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_post_nbr),
                         outputType=gdal.GDT_Int16)

post_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_nbr = ((pre_nbr  - post_nbr) * scaling_factor).astype(float)

post_nbr = None

In [ ]:
rbr = delta_nbr / (pre_nbr * scaling_factor + 1.001)

delta_nbr = pre_nbr = None

In [ ]:
temp_name = '_BURNED_RBR.tif'
output_name = 'BURNED_RBR.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(rbr),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [ ]:
results['B02'] = output_name

In [ ]:
catalog = Catalog(id='catalog',
                      description='Results')

catalog.clear_items()
catalog.clear_children()

In [ ]:
item_name = 'BURNED_AREA_DELINEATION'


pre_event_item.properties.pop('eo:bands', None)

result_item = Item(id=item_name,
                   geometry=pre_event_item.geometry,
                   bbox=pre_event_item.bbox,
                   datetime=pre_event_item.datetime,
                   properties=pre_event_item.properties)

result_item.common_metadata.set_gsd(10)

eo_item = extensions.eo.EOItemExt(result_item)

In [ ]:
results

In [ ]:
default_bands = {'B01':  {'name': 'NDVI/NDWI Threshold', 'common_name': 'tvi'},
                 'B02': {'name': 'Normalized Burn Ratio (NBR) ', 'common_name': 'dnbr'}}

In [ ]:
bands = []
   
os.makedirs(os.path.join('.', item_name),
                exist_ok=True)

for key, value in results.items():

    print(key, value)
    
    

    result_item.add_asset(key=key,
                          asset=Asset(href='./{}'.format(value), 
                                    media_type=MediaType.GEOTIFF))

    asset = result_item.get_assets()[key]                                   

    stac_band = extensions.eo.Band.create(name=key, 
                                           common_name=default_bands[key]['common_name'],
                                                description=default_bands[key]['name'])
    bands.append(stac_band)

    eo_item.set_bands([stac_band], asset=asset)

    shutil.move(value, os.path.join('.', item_name, value))
    
eo_item.set_bands(bands)

eo_item.apply(bands)  

### Results verification

In [ ]:
catalog.add_items([result_item])
    
catalog.normalize_and_save(root_href='./',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [ ]:
catalog.describe()

In [ ]:
item = next(catalog.get_items())

In [ ]:
item.get_assets()

In [ ]:
asset = item.get_assets()['B01']
asset.get_absolute_href()

In [ ]:
asset = item.get_assets()['B02']
asset.get_absolute_href()

In [ ]:
eo_item = extensions.eo.EOItemExt(item)
    
for index, band in enumerate(eo_item.bands):

    if band.common_name in ['tvi']:

        print(band.common_name, item.assets[band.name].get_absolute_href())
        
    if band.common_name in ['dnbr']:

        print(band.common_name, item.assets[band.name].get_absolute_href())